## 訓練データを.npyで保存

- 前回の議題の時に、データの読み込みがcolab上うまくいかないという話がありましたが、colab はIOが弱いため、データのロードに失敗していたようです。具体的には、一つのディレクトリに数千数万ものデータがあるとデータを読み込む際に、IO Error を吐いてしまうため、読みこめなかったようです。（うまく通るときもありますがガチャ）
- また、うまく通ったとしても、60000枚のデータを読み込むのは時間がかかるため、別の方法で読み込んだ方が効率が良いと考え、npyで保存することとしました。
- 以下、コードの例です。
    - こちらのコードはローカルで動かすことを推奨します。(Colabだとデータの読み込みが困難)
- より効果的な解決策がある場合は教えて頂きたいです！

In [ ]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [2]:
# tsvファイルの読み込み
train_master = pd.read_csv('train_master.tsv', sep='\t')
label_master = pd.read_csv('label_master.tsv', sep='\t')

master = pd.merge(train_master, label_master, on='label_id', how='inner')
sample = pd.read_csv("sample_submit.csv", header=None, sep=",")

In [3]:
master

,file_name,label_id,label_name
0,train_00000.png,11,large_omnivores_and_herbivores
1,train_00009.png,11,large_omnivores_and_herbivores
2,train_00034.png,11,large_omnivores_and_herbivores
3,train_00072.png,11,large_omnivores_and_herbivores
4,train_00083.png,11,large_omnivores_and_herbivores
...,...,...,...
49995,train_49683.png,0,aquatic_mammals
49996,train_49720.png,0,aquatic_mammals
49997,train_49779.png,0,aquatic_mammals
49998,train_49929.png,0,aquatic_mammals


In [ ]:
# このコードを実行するカレントディレクトリは"../inputs"を想定

DATADIR = "inputs"
training_data = []
test_data = []

def create_data(data):
    path = data
    if data == 'train':
        for image_name, class_num, _ in master.values:
            print(image_name)
            img_array = img_to_array(load_img(os.path.join(path, image_name)))  # 画像読み込み
            training_data.append([img_array, class_num])  # 画像データ、ラベル情報を追加
    if data == 'test':
        for image_name in sample[0].values:
            img_array = img_to_array(load_img(os.path.join(path, image_name))) # 画像読み込み            
            test_data.append(img_array)  # 画像データ、ラベル情報を追加
            
create_data('train')
create_data('test')

# 訓練データと検証データに8:2で分割
train_data, valid_data = train_test_split(training_data, shuffle=True, test_size=0.2, random_state=42)

# データセットの作成
X_train = []  # 画像データ
y_train = []  # ラベル情報

# データセット作成
for feature, label in train_data:
    X_train.append(feature)
    y_train.append(label)
# numpy配列に変換
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_valid = []  # 画像データ
y_valid = []  # ラベル情報

# データセット作成
for feature, label in valid_data:
    X_valid.append(feature)
    y_valid.append(label)
# numpy配列に変換
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

# numpy配列に変換
test_data = np.asarray(test_data)

In [8]:
# .npy に保存
# 保存したファイルをgoogle doraibuにアップロードしてご利用ください
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_valid', X_valid)
np.save('y_valid', y_valid)
np.save('test_data', test_data)

In [14]:
# データのロード
X_train = np.load('X_train')
y_train = np.load('y_train')

array([[[[199., 215., 249.],
         [196., 211., 244.],
         [195., 210., 243.],
         ...,
         [216., 231., 250.],
         [217., 231., 250.],
         [224., 234., 252.]],

        [[197., 210., 239.],
         [195., 208., 238.],
         [195., 210., 240.],
         ...,
         [231., 243., 250.],
         [233., 243., 250.],
         [241., 245., 253.]],

        [[222., 226., 246.],
         [213., 220., 242.],
         [209., 219., 243.],
         ...,
         [243., 250., 251.],
         [244., 249., 251.],
         [250., 250., 253.]],

        ...,

        [[ 72.,  73.,  99.],
         [ 71.,  74., 102.],
         [ 74.,  78., 108.],
         ...,
         [220., 208., 217.],
         [183., 168., 181.],
         [155., 141., 150.]],

        [[ 72.,  75., 104.],
         [ 76.,  81., 111.],
         [ 84.,  89., 122.],
         ...,
         [222., 212., 220.],
         [187., 174., 192.],
         [145., 132., 149.]],

        [[ 80.,  85., 118.],
       